# Bias of Anomaly Classification
In this notebook, we investigate the bias $B$ of the anomaly quantification for heat
flow that is gamma distributed through important parts of the $(\alpha,\beta)$ parameter
space. The bias $B(t)$ is the relative under- or overestimate of the heat power $P_H$ at
the fraction $r=t$ of evaluations of the posterior tail quantile $t$. If the posterior
estimate were "unbiased" in this sense, we would assume that in a fraction $t$ of samples,
the use of the tail quantile $t$ would lead to a power $P_H$ that is larger than the
posterior estimate. (I.e. this would be a coincidence of the credibility interval with a
frequentist exceedance rate)

In [ ]:
import numpy as np
from plotconfig import *
from cmcrameri.cm import *
from itertools import product
from cache import cached_call
from reheatfunq import GammaConjugatePrior
from reheatfunq.resilience import test_performance_cython
from matplotlib.colors import LogNorm, SymLogNorm, ListedColormap, rgb_to_hsv, hsv_to_rgb

In [ ]:
PMIN = 10.0
PMAX = 445e5 * 160e3 * 15e3 * 2.536783358701167e-09

In [ ]:
M = 1000

In [ ]:
PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V = np.loadtxt('results/05-GCP-Parameters.txt', skiprows=1, delimiter=',')

In [ ]:
BETA  = np.geomspace(2e-2, 15.0, 50)
ALPHA = np.geomspace(1, 1000, 51)

In [ ]:
ALPHA_fine = np.geomspace(1, 1000, 200)
BETA_fine = np.geomspace(2e-2, 15.0, 190)
Z_fine = np.zeros((ALPHA_fine.size, BETA_fine.size))
gcp = GammaConjugatePrior(PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)
for i,a in enumerate(ALPHA_fine):
    for j,b in enumerate(BETA_fine):
        Z_fine[i,j] = gcp.probability(np.atleast_1d(a), np.atleast_1d(b))

In [ ]:
QUANTILES = np.array([0.1])
N = 24
def evaluate_bias(N, ALPA, BETA, M, gcp, gcp_significance_level=1e-7, seed=289817):
    Z = np.zeros((ALPHA.size, BETA.size))
    ag,bg = np.meshgrid(ALPHA,BETA,indexing='ij')
    Z.flat = gcp.probability(ag.reshape(-1), bg.reshape(-1))
    gcp_significance_mask = Z >= gcp_significance_level
    res_bias = np.zeros((2,ALPHA.size,BETA.size,2))
    rng = np.random.default_rng(seed)
    seeds = rng.integers(2**63, size=2 * ALPHA.size * BETA.size)
    k = 0
    for p,PMW in enumerate([10.0, 271]):
        for i,j in product(range(ALPHA.size),range(BETA.size)):
            print("alpha:",ALPHA[i],",  beta:",BETA[j])
            if not gcp_significance_mask[i,j]:
                res_bias[p,i,j,:] = np.NaN
                continue

            res_ij = cached_call(test_performance_cython, np.array([N]), M, PMW, ALPHA[i], 1.0/BETA[j],
                                 QUANTILES, PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V, seed=seeds[k])[:,0,0,:]
            res_bias[p,i,j,0] = np.quantile(res_ij[0,:], 0.1)
            res_bias[p,i,j,1] = np.quantile(res_ij[1,:], 0.1)
            k += 1
    
    return res_bias

In [ ]:
res_bias = evaluate_bias(N, ALPHA, BETA, M, gcp)

Generate a shifted color map for the symmetric log-norm. We darken the minimum
(*h,s,v*)-value *v* to zero:

In [ ]:
cimg = 100*(1e-6*res_bias[0,:,:,0].T - [10.0, 271][0]) / [10.0, 271][0]
a = np.log10(cimg[~np.isnan(cimg)].max()) - np.log10(10.0) + 1.0
b = -cimg[~np.isnan(cimg)].min() / 10.0
c = 2*a
a /= c
b /= c

vik_colors = vik(np.linspace(0.5-b, 1.0, 1000))
vik_colors = rgb_to_hsv(vik_colors[:,:3])
vmax_old = vik_colors[:,2].max()
vik_colors[:,2] -= vik_colors[:,2].min()
vik_colors[:,2] *= vmax_old / vik_colors[:,2].max()
vik_colors = hsv_to_rgb(vik_colors)
vik_mod = ListedColormap(vik_colors)

In [ ]:
fig = plt.figure(dpi=140, figsize=(6.975, 2.8))
#ax_bg = fig.add_axes((0,0,1,1))
ax0 = fig.add_axes((0.063, 0.11, 0.39, 0.815))
ax1 = fig.add_axes((0.573, 0.11, 0.39, 0.815))

axtitle = ['63\,kW\,km^{-1}','1.7\,MW\,km^{-1}']

for i,ax in enumerate([ax0,ax1]):
    ax.set_xscale('log')
    ax.set_yscale('log')
    cimg = 100*(1e-6*res_bias[i,:,:,0].T - [10.0, 271][i]) / [10.0, 271][i]
    vmin = cimg[~np.isnan(cimg)].min()
    vmax = cimg[~np.isnan(cimg)].max()
    if i == 1:
        h = ax.pcolormesh(ALPHA, BETA, cimg, vmin=vmin, vmax=-vmin, cmap=vik, rasterized=True)
    else:
        h = ax.pcolormesh(ALPHA, BETA, cimg, cmap=vik_mod,
                          norm=SymLogNorm(10.0, vmin=vmin, vmax=vmax),
                          rasterized=True)
    
    cntr = ax.contour(ALPHA_fine, BETA_fine, Z_fine.T, linewidths=0.7, colors='k', levels=[1e-9, 1e-6, 1e-3, 1])
    ax.clabel(cntr, fmt=lambda x : f"$10^{{{round(np.log10(x))}}}$" if round(np.log10(x)) != 0 else 1,
              fontsize=6)
    ax.set_title(['(a)','(b)'][i] + f' $P_H/L = \\mathrm{{{axtitle[i]}}}$')
    ax.set_xlabel('$\\alpha$', labelpad=0.0)
    ax.set_ylabel('$\\beta$', labelpad=0.0)
    fig.colorbar(h, ax=ax, extend='max', label='Bias $B$ of 10 % tail quantile $P_H$ relative to\ntrue anomaly strength '
                                              f'$P_H={[10, 271][i]}\,\mathrm{{MW}}$ (%)');
fig.savefig('figures/A7-bias-10p-tail-gamma-with-prior.pdf')

### License
```
A notebook to compute the bias of the REHEATFUNQ heat flow anomaly
tail quantile for gamma-distributed heat flow and different heat
flow anomaly strengths.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```